# Agents to Research and Write an Article

Introduction to the foundational concepts of multi-agent systems and an overview of the crewAI framework.

# Using crewAI with MistralAI

In [21]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [22]:
!pip install crewai
!pip install crewai_tools
!pip install langchain_community

(3.8.2)


In [23]:
from crewai import Agent, Task, Crew

In [24]:
from google.colab import userdata
# userdata.get('OPEN_API_KEY')


In [25]:
import os
from google.colab import userdata  # Secure way to store API keys

# Set Mistral API credentials
os.environ["OPENAI_API_KEY"] = userdata.get('OPEN_API_KEY')  # Key saved in Colab secrets
os.environ["OPENAI_API_BASE"] = "https://api.mistral.ai/v1"
os.environ["OPENAI_MODEL_NAME"] = "mistral-small"  # or "mistral-medium"

### Agent: Planner

In [26]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
    verbose=True
)

### Agent: Writer

In [27]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [28]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

### Task: Plan

In [29]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [30]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [31]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Creation of crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.

In [32]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=True,
    manager_llm="mistral-small"
)

## Running the Crew

In [33]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 29ca71ba-8848-4afd-8a37-58b7da1f0831                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Provider List: https://docs.litellm.ai/docs/providers

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Content Planner                                                                                         │
│                                                                                                                 │
│  Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.            │
│  2. Identify the target audience, considering their interests and pain points.                                  │
│  3. Develop a detailed content outline including an introduction, key points, and a call to action.             │
│  4. Include SEO keywords and relevant data or sources.                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Content Planner                                                                                         │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Title: Artificial Intelligence: The Future of Technology and Its Impact on Businesses                          │
│                                                                                                                 │
│  Introduction:                                                                                                  │
│  Artificial Intelligence (AI) is rapidly transforming the way we live and work. From self-driving cars to       │
│  voice assistants, AI is becoming increasingly prevalent in our daily lives. This article will explore the      │
│  latest trends, key players, and noteworthy news in AI, and discuss how businesses can leverage this            │
│  technology to gain a competitive advantage.                                                                    │
│                                                                                                                 │
│  Target Audience:                                                                                               │
│  The target audience for this article includes business owners, executives, and professionals who are           │
│  interested in learning about AI and its potential impact on their industry. They may be struggling to keep up  │
│  with the rapid pace of technological change or seeking to improve their company's operational efficiency and   │
│  productivity.                                                                                                  │
│                                                                                                                 │
│  Content Outline:                                                                                               │
│                                                                                                                 │
│  I. Introduction                                                                                                │
│                                                                                                                 │
│  * Explanation of Artificial Intelligence                                                                       │
│  * Importance of staying up-to-date with AI trends                                                              │
│                                                                                                                 │
│  II. Latest Trends in AI                                                                                        │
│                                                                                                                 │
│  * Natural Language Processing (NLP)                                                                            │
│  * Computer Vision                                                                                              │
│  * Machine Learning (ML)                                                                                        │
│  * Robotic Process Automation (RPA)                                                                             │
│                                                                                                                 │
│  III. Key Players in AI                                                                                         │
│                                                        

...

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Editor                                                                                                  │
│                                                                                                                 │
│  Task: Proofread the given blog post for grammatical errors and alignment with the brand's voice.               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Editor                                                                                                  │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ------------------------------------------------                                                               │
│                                                                                                                 │
│  Artificial Intelligence: The Future of Technology and Its Impact on Businesses                                 │
│  =============================================================================                                  │
│                                                                                                                 │
│  Artificial Intelligence (AI) is rapidly transforming the way we live and work. From self-driving cars to       │
│  voice assistants, AI is becoming increasingly prevalent in our daily lives. This article will explore the      │
│  latest trends, key players, and noteworthy news in AI, and discuss how businesses can leverage this            │
│  technology to gain a competitive advantage.                                                                    │
│                                                                                                                 │
│  Explanation of Artificial Intelligence                                                                         │
│  ---------------------------------------                                                                        │
│                                                                                                                 │
│  Artificial Intelligence (AI) refers to the simulation of human intelligence in machines that are programmed    │
│  to think like humans and mimic their actions. The term may also be applied to any machine that exhibits        │
│  traits associated with a human mind such as learning and problem-solving. AI is not just a technology but a    │
│  collection of many technologies working together to enable machines to sense, comprehend, act and learn,       │
│  thereby improving over time.                                                                                   │
│                                                                                                                 │
│  Importance of staying up-to-date with AI trends                                                                │
│  -------------------------------------------------                                                              │
│                                                                                                                 │
│  With the rapid pace of technological change, it's important for businesses to stay up-to-date with the latest  │
│  AI trends in order to remain competitive. By understanding and implementing the latest AI technologies,        │
│  businesses can improve their operational efficiency and productivity, enhance the customer experience and      │
│  engagement, and drive innovation and differentiation. Moreover, AI can help businesses to make better          │
│  decisions, predict trends, and identify new opportunities.                                                     │
│                                                                                                                 │
│  Latest Trends in AI                                                                                            │
│  -------------------                                   

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: bd11e477-9045-440b-b0b8-d415bb737c89                                                                     │
│  Agent: Editor                                                                                                  │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 29ca71ba-8848-4afd-8a37-58b7da1f0831                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: ------------------------------------------------                                                 │
│                                                                                                                 │
│  Artificial Intelligence: The Future of Technology and Its Impact on Businesses                                 │
│  =============================================================================                                  │
│                                                                                                                 │
│  Artificial Intelligence (AI) is rapidly transforming the way we live and work. From self-driving cars to       │
│  voice assistants, AI is becoming increasingly prevalent in our daily lives. This article will explore the      │
│  latest trends, key players, and noteworthy news in AI, and discuss how businesses can leverage this            │
│  technology to gain a competitive advantage.                                                                    │
│                                                                                                                 │
│  Explanation of Artificial Intelligence                                                                         │
│  ---------------------------------------                                                                        │
│                                                                                                                 │
│  Artificial Intelligence (AI) refers to the simulation of human intelligence in machines that are programmed    │
│  to think like humans and mimic their actions. The term may also be applied to any machine that exhibits        │
│  traits associated with a human mind such as learning and problem-solving. AI is not just a technology but a    │
│  collection of many technologies working together to enable machines to sense, comprehend, act and learn,       │
│  thereby improving over time.                                                                                   │
│                                                                                                                 │
│  Importance of staying up-to-date with AI trends                                                                │
│  -------------------------------------------------                                                              │
│                                                                                                                 │
│  With the rapid pace of technological change, it's important for businesses to stay up-to-date with the latest  │
│  AI trends in order to remain competitive. By understanding and implementing the latest AI technologies,        │
│  businesses can improve their operational efficiency and productivity, enhance the customer experience and      │
│  engagement, and drive innovation and differentiation. Moreover, AI can help businesses to make better          │
│  decisions, predict trends, and identify new opportunities.                                                     │
│                                                                                                                 │
│  Latest Trends in AI                                  

- Display the results of your execution as markdown in the notebook.

In [34]:
from IPython.display import Markdown
# Markdown(result)
Markdown(result.raw)

------------------------------------------------

Artificial Intelligence: The Future of Technology and Its Impact on Businesses
=============================================================================

Artificial Intelligence (AI) is rapidly transforming the way we live and work. From self-driving cars to voice assistants, AI is becoming increasingly prevalent in our daily lives. This article will explore the latest trends, key players, and noteworthy news in AI, and discuss how businesses can leverage this technology to gain a competitive advantage.

Explanation of Artificial Intelligence
---------------------------------------

Artificial Intelligence (AI) refers to the simulation of human intelligence in machines that are programmed to think like humans and mimic their actions. The term may also be applied to any machine that exhibits traits associated with a human mind such as learning and problem-solving. AI is not just a technology but a collection of many technologies working together to enable machines to sense, comprehend, act and learn, thereby improving over time.

Importance of staying up-to-date with AI trends
-------------------------------------------------

With the rapid pace of technological change, it's important for businesses to stay up-to-date with the latest AI trends in order to remain competitive. By understanding and implementing the latest AI technologies, businesses can improve their operational efficiency and productivity, enhance the customer experience and engagement, and drive innovation and differentiation. Moreover, AI can help businesses to make better decisions, predict trends, and identify new opportunities.

Latest Trends in AI
-------------------

There are several key AI trends that businesses should be aware of, including:

### Natural Language Processing (NLP)

NLP is a branch of AI that deals with the interaction between computers and humans through natural language. NLP allows machines to understand human language, interpret it, and generate a response in a valuable way. NLP is used in various applications such as virtual assistants, chatbots, and language translation tools.

### Computer Vision

Computer vision is the ability of a machine to interpret and understand the visual world. With computer vision, machines can accurately identify and classify objects and then react to what they "see." Computer vision is used in various applications such as facial recognition, autonomous vehicles, and medical imaging.

### Machine Learning (ML)

Machine learning is a type of AI that allows machines to learn and adapt without being explicitly programmed. By using algorithms that can learn from and make decisions or predictions based on data, machine learning allows machines to improve their performance on a specific task over time. Machine learning is used in various applications such as fraud detection, recommendation engines, and predictive maintenance.

### Robotic Process Automation (RPA)

RPA is a type of AI that automates repetitive tasks by replicating the actions of a human user. By using RPA, businesses can automate time-consuming and rule-based tasks, freeing up employees to focus on more strategic and value-added activities. RPA is used in various applications such as finance, human resources, and customer service.

Key Players in AI
----------------

There are several key players in the AI industry, including:

### Google

Google is one of the leading companies in the AI field, with a strong focus on natural language processing and computer vision. Google's AI platform, TensorFlow, is one of the most popular open-source machine learning frameworks. Google is using AI to improve its search engine, Gmail, Google Photos, and Google Assistant.

### Amazon

Amazon is another major player in the AI industry, with a focus on machine learning and natural language processing. Amazon's AI assistant, Alexa, is one of the most popular virtual assistants on the market. Amazon is using AI to improve its e-commerce platform, recommendation engine, and logistics operations.

### Microsoft

Microsoft is a major player in the AI industry, with a focus on machine learning and cognitive services. Microsoft's Azure AI platform is a comprehensive set of cloud-based AI services that allow developers to build intelligent applications. Microsoft is using AI to improve its productivity tools, gaming platform, and customer service operations.

### IBM

IBM is a leader in the AI industry, with a focus on machine learning and natural language processing. IBM's Watson is one of the most advanced AI platforms, with a wide range of applications in industries such as healthcare, finance, and customer service. IBM is using AI to improve its cloud platform, security services, and supply chain operations.

Noteworthy News in AI
---------------------

There are several noteworthy news in AI, including:

### Ethical concerns and regulations

As AI becomes more prevalent, there are growing ethical concerns and regulations surrounding its use. Governments and organizations are beginning to develop regulations around AI to ensure that it is used ethically and responsibly. There are concerns around bias, privacy, transparency, and accountability.

### AI in healthcare

AI is being increasingly used in healthcare to improve patient outcomes and reduce costs. AI is being used to diagnose diseases, develop new treatments, and personalize patient care. AI is also being used to improve medical imaging, drug discovery, and clinical trials.

### AI in finance

AI is being increasingly used in finance to improve operational efficiency and reduce costs. AI is being used to automate time-consuming tasks, detect fraud, and provide personalized financial advice. AI is also being used to improve risk management, credit decisioning, and investment management.

How Businesses Can Leverage AI
-----------------------------

Businesses can leverage AI in several ways, including:

### Operational efficiency and productivity

AI can help businesses improve their operational efficiency and productivity by automating repetitive tasks, freeing up employees to focus on more strategic and value-added activities. AI can also help businesses to optimize their supply chain, logistics, and inventory management operations.

### Customer experience and engagement

AI can help businesses enhance the customer experience and engagement by providing personalized recommendations, predicting customer behavior, and automating customer service. AI can also help businesses to improve their marketing campaigns, social media engagement, and customer feedback analysis.

### Innovation and differentiation

AI can help businesses drive innovation and differentiation by providing new insights, identifying new opportunities, and enabling new business models. AI can also help businesses to develop new products and services, improve their research and development operations, and enhance their data analytics capabilities.

Encourage businesses to explore AI solutions
--------------------------------------------

With the potential benefits that AI can bring, it's essential for businesses to explore AI solutions. By implementing AI, businesses can gain a competitive advantage and improve their bottom line. Businesses should start by identifying their use cases, assessing their data readiness, and selecting the right AI technology and partner.

Offer further resources and assistance
---------------------------------------

There are several resources and assistance available to help businesses get started with AI. These include online courses, consultants, and AI platforms. Businesses can also join AI communities, attend AI events, and collaborate with other businesses and startups.

Conclusion
----------

Artificial Intelligence is rapidly transforming the way we live and work, and businesses that fail to keep up with the latest AI trends risk being left behind. By understanding and implementing the latest AI technologies, businesses can improve their operational efficiency and productivity, enhance the customer experience and engagement, and drive innovation and differentiation.

In this article, we have explored the latest trends, key players, and noteworthy news in AI. We have also discussed how businesses can leverage this technology to gain a competitive advantage.

By staying up-to-date with the latest AI trends, businesses can position themselves to take advantage of the many benefits that AI has to offer.

*Explore AI solutions for your business today.*

------------------------------------------------

References:

* Forbes: "The 10 Most Innovative Companies In Artificial Intelligence"
* Deloitte: "The State of AI in the Enterprise"
* McKinsey: "Artificial Intelligence: The Next Digital Frontier?"
* Harvard Business Review: "The Ethics of Artificial Intelligence"
* Gartner: "Emerging Technologies and Trends Impact Radar: Artificial Intelligence"

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

# Using GigaChat